In [161]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

result_dir = os.path.join(os.getcwd(), "results_True")
results = os.listdir(result_dir)


def load(f):
    file_dir = os.path.join(result_dir, f, "results.json")
    with open(file_dir, "r") as f:

        results = json.load(f)
    return results


# create a function that captalizes the first letter of a word
def capitalize(string):
    return string[0].upper() + string[1:]


def load_from_query(query):
    results = []
    for f in os.listdir(result_dir):
        if f.find(query) != -1:
            return load(f)
    return None


def show(data, scale=1, labels=None):
    for i, d in enumerate(data):
        print(
            ("" if not labels else labels[i])
            + " "
            + "|".join([f"{x*scale if x != None else -1:.4f}" for x in d])
        )

    # Create a folder called "csv" if it doesn't exist
    csv_dir = os.path.join(os.getcwd(), "csv")
    os.makedirs(csv_dir, exist_ok=True)

    # Write the data to CSV files in the "csv" folder
    pd.DataFrame(avg_delay, index=labels, columns=cache_sizes).to_csv(
        os.path.join(csv_dir, "avg_delay.csv")
    )
    pd.DataFrame(hit_ratio, index=labels, columns=cache_sizes).to_csv(
        os.path.join(csv_dir, "hit_ratio.csv")
    )
    pd.DataFrame(success_ratio, index=labels, columns=cache_sizes).to_csv(
        os.path.join(csv_dir, "success_ratio.csv")
    )


cache_sizes = [
    50,
    75,
    100,
    125,
    150,
    175,
    200,
]
NR = 3
Nr = 30

avg_delay = []
hit_ratio = []
success_ratio = []
labels = []

for cache in [
    "random",
    "mcfed",
    "avgfed",
    "nocache",
]:
    for delivery in ["random", "greedy", "drl", "norsu"]:
        if cache in ["random", "avgfed", "nocache"] and delivery not in [
            "drl",
            "norsu",
        ]:
            continue
        if delivery == "norsu" and cache != "random":
            continue

        labels.append(f"{capitalize(cache)}_{capitalize(delivery)}")
        sub_data1 = []
        sub_data2 = []
        sub_data3 = []
        for cache_size in cache_sizes:
            query = f"{NR}_{cache_size}_{Nr}_{cache}_{delivery}"
            result = load_from_query(query)
            if result:
                sub_data1.append(np.mean(result["round_avg_delay_tracking"]))
                sub_data2.append(np.mean(result["round_hit_ratio_tracking"]))
                sub_data3.append(np.mean(result["round_success_ratio_tracking"]))
            else:
                sub_data1.append(None)
                sub_data2.append(None)
                sub_data3.append(None)
        avg_delay.append(sub_data1)
        hit_ratio.append(sub_data2)
        success_ratio.append(sub_data3)

In [162]:
labels

['Random_Drl',
 'Random_Norsu',
 'Mcfed_Random',
 'Mcfed_Greedy',
 'Mcfed_Drl',
 'Avgfed_Drl',
 'Nocache_Drl']

In [163]:
show(avg_delay, 1000, labels)

Random_Drl 280.2540|272.8095|272.5214|270.7990|270.2114|267.8026|277.6809
Random_Norsu 365.0044|364.3861|363.7089|363.7208|365.4846|364.5453|363.5685
Mcfed_Random 266.7606|265.4980|262.3030|265.0022|261.4415|262.1577|260.4523
Mcfed_Greedy 272.6803|259.9850|257.9839|250.3510|244.5455|238.8611|237.9841
Mcfed_Drl 264.8744|267.3753|261.3971|262.8085|266.6302|263.0632|259.9954
Avgfed_Drl 279.9762|266.3984|268.5590|265.4123|267.1973|273.3872|263.0606
Nocache_Drl 276.5774|272.8644|275.4563|274.0449|271.1295|272.4993|272.5234


In [164]:
show(hit_ratio, 100, labels)

Random_Drl 0.8111|1.3111|1.5778|2.1378|2.5000|3.1444|3.0689
Random_Norsu 0.0000|0.0000|0.0000|0.0000|0.0000|0.0000|0.0000
Mcfed_Random 5.5356|8.0511|10.7711|10.8267|12.7111|13.5333|15.3356
Mcfed_Greedy 19.5644|28.6378|29.7156|35.1378|38.5133|43.6533|44.9667
Mcfed_Drl 7.8133|8.7800|17.6400|13.2889|12.4244|15.6533|14.9489
Avgfed_Drl 4.5756|7.0333|8.3600|9.3044|10.8867|9.9711|12.1022
Nocache_Drl 0.0000|0.0000|0.0000|0.0000|0.0000|0.0000|0.0000


In [165]:
show(success_ratio, 100, labels)

Random_Drl 47.4044|51.1978|51.8844|51.9933|52.0289|53.2511|47.5111
Random_Norsu 100.0000|100.0000|100.0000|100.0000|100.0000|100.0000|100.0000
Mcfed_Random 74.5133|74.6622|75.4644|75.0667|75.6311|75.8911|75.6489
Mcfed_Greedy 72.0000|73.0067|70.2733|64.4044|65.6400|63.8000|59.7622
Mcfed_Drl 52.6778|51.4156|73.9267|51.7111|50.4756|51.1889|52.5622
Avgfed_Drl 46.1711|52.9689|50.7933|52.5156|50.3489|47.4400|52.4867
Nocache_Drl 49.5800|51.8244|49.9267|51.0689|52.3089|51.7044|52.2778
